# Lab03 
## Forecasting Several Time Steps Ahead
- RNN으로 여러개를 예측하기 (loop_forward)

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys

import time
import myplot

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
print("python", sys.version)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)    
    
%matplotlib inline

C:\Users\admin\Anaconda3\envs\tf2.5\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\admin\Anaconda3\envs\tf2.5\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\admin\Anaconda3\envs\tf2.5\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


python 3.8.0 (default, Nov  6 2019, 16:00:02) [MSC v.1916 64 bit (AMD64)]
matplotlib 3.4.2
numpy 1.19.5
pandas 1.3.2
sklearn 0.24.2
tensorflow 2.5.0
tensorflow.keras 2.5.0


In [ ]:
def generate_time_series(batch_size, n_steps):        
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    
    time = np.linspace(0, 1, n_steps)    
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2    
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise    
    return series[..., np.newaxis].astype(np.float32)

## b3. RNN (10 Steps Ahead that predicts the next 10 steps at each time step)

In [ ]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(1000, n_steps + 10)
X_train = series[:700, :n_steps]
X_valid = series[700:900, :n_steps]
X_test = series[900:, :n_steps]
Y = np.empty((1000, n_steps, 10))

In [ ]:
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
    
Y_train = Y[:700]
Y_valid = Y[700:900]
Y_test = Y[900:]

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

simpleRNN_TimeD = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [ ]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

In [ ]:
model = simpleRNN_TimeD

In [ ]:
opt = keras.optimizers.Adam(lr=0.01)
model.compile(loss="mse", optimizer=opt, metrics=[last_time_step_mse])
model.summary()

In [ ]:
plot_model(model, to_file='./b3_model_simpleRNN_TimeD.png', show_shapes=True)

In [ ]:
%%time
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

In [ ]:
myplot.plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, 50:, :]
Y_pred = model.predict(X_new)[:, -1][..., np.newaxis]

myplot.plot_multiple_forecasts(X_new, Y_new, Y_pred)
plt.show()

In [ ]:
b3 = history.history['loss'][np.argmin(history.history['loss'])]
print('The loss of SimpleRNN_TimeD', b3)

## b4. Deep RNN with Batch Norm

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

deepRNN_TimeD = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.BatchNormalization(),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model = deepRNN_TimeD
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
model.summary()

In [ ]:
plot_model(model, to_file='./b4_model_deepRNN_TimeD.png', show_shapes=True)

In [ ]:

%%time
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

In [ ]:
myplot.plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
np.random.seed(43)

series = generate_time_series(1, 50 + 10)
X_new, Y_new = series[:, :50, :], series[:, 50:, :]
Y_pred = model.predict(X_new)[:, -1][..., np.newaxis]

myplot.plot_multiple_forecasts(X_new, Y_new, Y_pred)
plt.show()

In [ ]:
b4 = history.history['loss'][np.argmin(history.history['loss'])]
print('The loss of DeepRNN_TimeD:', b4)

# Results Summary

In [ ]:
models = pd.DataFrame({
    'Model': ['simpleRNN_TimeD', 'deepRNN_TimeD'],
    'Score': [b3, b4]})
models.sort_values(by='Score', ascending=True)